In [1]:
! ls /scratch/Projects/IMMUNEX/segmentation/bin2cell/

bin2cell_output  bin2cell_output_he0.01_gex0.05  bin2cell_output_he0005_gex005


In [2]:
import sys
sys.path.append("/opt/Banksy_py")
import banksy as banksy
import time; import random

from banksy.initialize_banksy import initialize_banksy
from banksy.run_banksy import run_banksy_multiparam
from banksy_utils.color_lists import spagcn_color
from banksy_utils.plot_utils import plot_qc_hist, plot_cell_positions
from banksy_utils.load_data import load_adata, display_adata
from banksy_utils.filter_utils import filter_cells
from banksy_utils.filter_utils import normalize_total, filter_hvg, print_max_min

/opt/Banksy_py/banksy/cluster_methods.py:18: UserWarning: 
No rpy2 installed. BANKSY will run, but mclust will not work.
Note: you can still use the default leiden option for clustering.
Install rpy2 and R in your conda environment if you want to use mclust for clustering.

  warnings.warn(warn_str)
2025-07-10 22:48:31.879986: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-10 22:48:31.889659: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-10 22:48:31.901504: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already 

In [3]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm
from anndata import AnnData
import scipy.sparse as sp

# Load the data
sample_id = "IMMUNEX002"

adata = sc.read(f"/scratch/Projects/IMMUNEX/segmentation/bin2cell/bin2cell_output_he0.01_gex0.05/{sample_id}/adata_processed.h5ad")
adata.raw = adata
adata.uns['sample_id'] = sample_id
adata

AnnData object with n_obs × n_vars = 10287064 × 18270
    obs: 'in_tissue', 'array_row', 'array_col', 'n_counts', 'destripe_factor', 'n_counts_adjusted', 'labels_he', 'labels_he_expanded', 'labels_gex', 'labels_joint', 'labels_joint_source'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'
    uns: 'bin2cell', 'spatial', 'sample_id'
    obsm: 'spatial', 'spatial_cropped_150_buffer'

In [4]:
# Load the metadata
metadata = pd.read_csv("~/rawdata/IMMUNEX/data/VisiumHD_18_2024_NSCLC.csv")

metadata_sample = metadata[metadata['Sample_code'] == sample_id]
adata.uns["sample_metadata"] = metadata_sample.iloc[0].to_dict()

display(adata.obs.head()) # Display the first few rows of the observation data (cell metadata)

,in_tissue,array_row,array_col,n_counts,destripe_factor,n_counts_adjusted,labels_he,labels_he_expanded,labels_gex,labels_joint,labels_joint_source
s_002um_02587_02503-1,1,2587,2503,28.0,0.823529,21.411765,0,123152,22606,123152,primary
s_002um_01674_00710-1,1,1674,710,2.0,0.233977,6.083413,0,168030,34839,168030,primary
s_002um_02498_02808-1,1,2498,2808,4.0,0.315715,8.208591,119256,119256,0,119256,primary
s_002um_00952_00136-1,1,952,136,1.0,0.143678,3.735632,330594,330594,0,330594,primary
s_002um_00564_00910-1,1,564,910,5.0,0.528161,13.732175,0,291153,0,291153,primary


In [5]:
# Sampling adata 
# Set seed for reproducibility
np.random.seed(42)

# Compute number of cells to sample
n_obs = adata.n_obs
n_sample = int(n_obs * 0.01)
n_sample = int(n_obs * 0.15)

# Randomly choose indices
sample_indices = np.random.choice(n_obs, size=n_sample, replace=False)

# Subset the AnnData object
adata_sampled = adata[sample_indices].copy()

# Inspect new shape
print(adata_sampled)


AnnData object with n_obs × n_vars = 1543059 × 18270
    obs: 'in_tissue', 'array_row', 'array_col', 'n_counts', 'destripe_factor', 'n_counts_adjusted', 'labels_he', 'labels_he_expanded', 'labels_gex', 'labels_joint', 'labels_joint_source'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'
    uns: 'bin2cell', 'spatial', 'sample_id', 'sample_metadata'
    obsm: 'spatial', 'spatial_cropped_150_buffer'


In [ ]:
adata_sampled.obs['cell_id'] = adata_sampled.obs['labels_joint'].astype(str)
adata_cells = adata_sampled[adata_sampled.obs['labels_joint'] > 0].copy()
adata_cells.var['mt'] = adata_sampled.var_names.str.upper().str.startswith('MT-')

# Ensure matrix is dense or convert to DataFrame
X = adata_cells.X.toarray() if sp.issparse(adata_cells.X) else adata_cells.X
df = pd.DataFrame(X, columns=adata_cells.var_names, index=adata_cells.obs['cell_id'])

# Compute mean expression per cell_id
grouped_expr = df.groupby(level=0).mean()

# Create a new AnnData object
adata_cells_mean = AnnData(X=sp.csr_matrix(grouped_expr.values))
adata_cells_mean.obs_names = grouped_expr.index
adata_cells_mean.var_names = grouped_expr.columns

adata_cells_mean.shape


In [ ]:
# and 'cell_id' is the grouping key used to generate grouped_expr
# 1. Get a mapping from cell_id → average spatial position
cell_coords = adata_sampled.obs[['cell_id']].copy()
cell_coords['x'] = adata_sampled.obsm['spatial'][:, 0]
cell_coords['y'] = adata_sampled.obsm['spatial'][:, 1]

# 2. Average spatial position per cell
mean_coords = cell_coords.groupby('cell_id')[['x', 'y']].mean()

# 3. Match to your grouped_expr.index (cell IDs)
mean_coords = mean_coords.loc[grouped_expr.index]

# 4. Assign to obsm
adata_cells_mean.obsm['spatial'] = mean_coords.values

# Copy the spatial dictionary to cdata
library_id = list(adata_sampled.uns['spatial'].keys())[0]
adata_cells_mean.uns['spatial'] = {
    library_id: adata_sampled.uns['spatial'][library_id]
}
adata_cells = adata_cells_mean

In [ ]:
# Add metadata
# Step 1: Copy gene metadata (var) for the grouped expression matrix
adata_cells.var = adata_sampled.var.loc[grouped_expr.columns].copy()
adata_cells.var_names_make_unique()

# Step 2: Aggregate metadata per reconstructed cell
meta = adata_sampled.obs.groupby('cell_id').agg({
    'n_counts': 'mean',
})
# - Number of bins per cell (count of occurrences)
meta['n_bins'] = adata_sampled.obs['cell_id'].value_counts()
meta.index.name = 'cell_id'

# Step 3: Reorder metadata to match cell ordering in grouped_expr
meta = meta.loc[grouped_expr.index]

# Step 4: Assign aggregated metadata as the new obs for reconstructed cells
adata_cells.obs = meta

# Step 5: Copy over select entries from uns
for k in ['sample_id', 'sample_metadata', 'bin2cell']:
    if k in adata.uns:
        adata_cells.uns[k] = adata.uns[k]

# Step 6: Copy full spatial metadata
adata_cells.uns['spatial'] = adata.uns['spatial']

# Done
adata_cells


## Cropped viz

In [ ]:
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt

crop_axis_fraction = 0.05  # how much to crop around the center

# Get spatial coordinates
spatial_coords = adata_cells.obsm['spatial'].copy()

# Compute coordinate bounds and center
x_min, x_max = np.min(spatial_coords[:, 0]), np.max(spatial_coords[:, 0])
y_min, y_max = np.min(spatial_coords[:, 1]), np.max(spatial_coords[:, 1])
center_x = (x_min + x_max) / 2
center_y = (y_min + y_max) / 2

# Define crop bounds around center
x0 = center_x - (x_max - x_min) * crop_axis_fraction / 2
x1 = center_x + (x_max - x_min) * crop_axis_fraction / 2
y0 = center_y - (y_max - y_min) * crop_axis_fraction / 2
y1 = center_y + (y_max - y_min) * crop_axis_fraction / 2

# Find cells within crop
within_crop = (
    (spatial_coords[:, 0] >= x0) & (spatial_coords[:, 0] <= x1) &
    (spatial_coords[:, 1] >= y0) & (spatial_coords[:, 1] <= y1)
)
adata_cropped = adata_cells[within_crop, :].copy()

# Normalize n_bins to define spot sizes (you can adjust scaling)
sizes = adata_cropped.obs['n_bins']
sizes_normalized = 10 * (sizes / sizes.max())  # scale to 0–20

# Plot
sc.pl.spatial(
    adata_cropped,
    color=['n_counts',None],
    size=sizes_normalized,
    cmap='Spectral_r',
    alpha_img=0.6,
    show=True
)


In [ ]:
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt

crop_axis_fraction = 0.33  # how much to crop around the center

# Get spatial coordinates
spatial_coords = adata_cells.obsm['spatial'].copy()

# Compute coordinate bounds and center
x_min, x_max = np.min(spatial_coords[:, 0]), np.max(spatial_coords[:, 0])
y_min, y_max = np.min(spatial_coords[:, 1]), np.max(spatial_coords[:, 1])
center_x = (x_min + x_max) / 2
center_y = (y_min + y_max) / 2

# Define crop bounds around center
x0 = center_x - (x_max - x_min) * crop_axis_fraction / 2
x1 = center_x + (x_max - x_min) * crop_axis_fraction / 2
y0 = center_y - (y_max - y_min) * crop_axis_fraction / 2
y1 = center_y + (y_max - y_min) * crop_axis_fraction / 2

# Find cells within crop
within_crop = (
    (spatial_coords[:, 0] >= x0) & (spatial_coords[:, 0] <= x1) &
    (spatial_coords[:, 1] >= y0) & (spatial_coords[:, 1] <= y1)
)
adata_cropped = adata_cells[within_crop, :].copy()

# Normalize n_bins to define spot sizes (you can adjust scaling)
sizes = adata_cropped.obs['n_bins']
sizes_normalized = 50 * (sizes / sizes.max())  # scale to 0–20

# Plot
sc.pl.spatial(
    adata_cropped,
    color=['n_counts',None],
    size=sizes_normalized,
    cmap='Spectral_r',
    alpha_img=0.6,
    show=True
)


In [ ]:
sc.pp.calculate_qc_metrics(adata_cells, inplace=True)
adata_cells.obs['n_bins'] = adata_cells.obs['n_bins'].astype(float)
sns.scatterplot(x=adata_cells.obs['n_bins'], y=adata_cells.obs['total_counts'])
plt.show()


In [ ]:
adata_cells

## QC Adata

In [ ]:
import sys
sys.path.append("/opt/Banksy_py")
import banksy as banksy
import time; import random

from banksy.initialize_banksy import initialize_banksy
from banksy.run_banksy import run_banksy_multiparam
from banksy_utils.color_lists import spagcn_color
from banksy_utils.plot_utils import plot_qc_hist, plot_cell_positions
from banksy_utils.load_data import load_adata, display_adata
from banksy_utils.filter_utils import filter_cells
from banksy_utils.filter_utils import normalize_total, filter_hvg, print_max_min

import scipy.sparse
if not scipy.sparse.issparse(adata_cells.X):
    print('converting to sparse')
    adata_cells.X = scipy.sparse.csr_matrix(adata_cells.X)
    # adata.raw = adata.X
display_adata(adata_cells)


In [ ]:
# QC
adata_cells.var_names_make_unique()
adata_cells.var["mt"] = adata_cells.var_names.str.startswith("MT-")

sc.pp.calculate_qc_metrics(adata_cells, 
                           qc_vars=["mt"], 
                           log1p=True, 
                           inplace=True)

# bin options for fomratting histograms
# Here, we set 'auto' for 1st figure, 80 bins for 2nd figure. and so on
hist_bin_options = ['auto', 50, 80, 100]

plot_qc_hist(adata_cells, 
         total_counts_cutoff = 800, # for visualization
         n_genes_high_cutoff = 1000, # for visualization
         n_genes_low_cutoff = 0, # for visualization
         bin_options = hist_bin_options)

In [ ]:
# initial filter of cells by their cell counts, MT count and gene counts
adata_filtred = adata_cells.copy()

adata_filtred = filter_cells(adata_filtred, 
             min_count=3, 
             max_count=500, 
             MT_filter=20, 
             gene_filter=100)

plot_qc_hist(adata_filtred,
         total_counts_cutoff = 800, # for visualization
         n_genes_high_cutoff = 1000, # for visualization
         n_genes_low_cutoff = 0, # for visualization
        bin_options = hist_bin_options)


## Normalize and scale

In [ ]:
#Identify highly variable genes — modify filter_hvg to skip log1p inside
adata_normalized = adata_filtred.copy()
adata_normalized = normalize_total(adata_normalized)
sc.pp.log1p(adata_normalized)


In [ ]:

sc.pp.highly_variable_genes(adata_normalized, n_top_genes=5000, flavor='seurat')

nsclc_marker_genes = sorted(list(set([
    # T cells
    'CD3D', 'CD3E', 'CD3G', 'CD2', 'CD7', 'TRAC', 'JUNB', 'S100A4', 'CD52', 'PFN1P1',
    'CD81', 'EEF1B2P3', 'CXCR4', 'CREM', 'IL32', 'TGIF1',

    # CD4+ T cells
    'CD4', 'IL7R', 'CCR7', 'SELL', 'FOXP3', 'CTLA4', 'TIGIT',

    # CD8+ T cells
    'CD8A', 'CD8B', 'GZMA', 'GZMB', 'PRF1', 'LAG3', 'PDCD1',

    # Regulatory T cells
    'IL2RA', 'IKZF2',

    # B cells
    'CD19', 'MS4A1', 'CD79A', 'CD79B', 'IGHD', 'IGHM', 'CD22', 'CD24',
    'CD74', 'HMGA1', 'CD52', 'PTPRC', 'HLA-DRA', 'CXCR4', 'SPCS3', 'LTB', 'IGKC',

    # Plasma cells
    'MZB1', 'JCHAIN', 'SDC1', 'XBP1',

    # NK cells
    'NCAM1', 'KLRB1', 'KLRD1', 'NKG7', 'GNLY', 'PRF1', 'FGFBP2',
    'IL32', 'FHL2', 'IL2RG', 'CD69', 'HOPX',

    # Dendritic cells
    'ITGAX', 'CD1C', 'CLEC9A', 'CD83', 'LAMP3', 'CCR7', 'HLA-DRA', 'BATF3',

    # Monocytes / Macrophages
    'CD14', 'CD68', 'LYZ', 'FCN1', 'S100A8', 'S100A9', 'ITGAM', 'MRC1', 'CD163',
    'NOS2', 'IL1B', 'TNF', 'ARG1', 'CD206', 'IL10',

    # Neutrophils
    'FCGR3B', 'CSF3R', 'ELANE', 'MPO', 'CEACAM8',

    # Endothelial
    'PECAM1', 'VWF', 'CDH5', 'KDR', 'CLDN5', 'ESAM', 'ENG', 'CD34', 'PROM1', 'PDPN',
    'TEK', 'FLT1', 'VCAM1', 'PTPRC', 'MCAM', 'ICAM1', 'FLT4',

    # Fibroblasts / CAFs
    'PDGFRA', 'PDGFRB', 'COL1A1', 'COL1A2', 'COL3A1', 'COL5A2', 'ACTA2', 'TAGLN', 'FAP', 'POSTN',
    'TCF21', 'FN',

    # Pericytes / Smooth muscle
    'RGS5', 'MYL9', 'MYLK', 'FHL2', 'ITGA1', 'EHD2', 'OGN', 'SNCG', 'FABP4',

    # Epithelial / Alveolar / Cancer cells
    'EPCAM', 'KRT8', 'KRT18', 'KRT19', 'SFTPC', 'SFTPB', 'AGER', 'PDPN', 'KRT7', 'KRT5', 'KRT14',
    'TP63', 'NGFR', 'MUC1', 'MUC16', 'BIRC5', 'MYC', 'SOX2', 'CCND1', 'CDKN2A', 'PD-L1',
    'EGFR', 'KRAS', 'TP53', 'TTF-1', 'NKX2-1', 'CEACAM5', 'CDH1', 'CLDN1',

    # Club/secretory cells
    'SCGB1A1', 'SCGB3A2', 'MUC5B',

    # Ciliated cells
    'FOXJ1', 'TPPP3', 'PIFO', 'DNAH5',

    # Goblet cells
    'MANF', 'KRT7', 'AQP3', 'AGR2', 'BACE2', 'TFF3', 'PHGR1', 'MUC4', 'MUC13', 'GUCA2A',

    # Enterocytes
    'CD55', 'ELF3', 'PLIN2', 'GSTM3', 'KLF5', 'CBR1', 'APOA1', 'CA1', 'PDHA1', 'EHF',

    # Enteroendocrine
    'NUCB2', 'FABP5', 'CPE', 'ALCAM', 'GCG', 'SST', 'CHGB', 'IAPP', 'CHGA', 'ENPP2',

    # Crypt cells / stem-like
    'HOPX', 'SLC12A2', 'MSI1', 'SMOC2', 'OLFM4', 'ASCL2', 'PROM1', 'BMI1', 'EPHB2', 'LRIG1',

    # Cancer stem cells
    'CD44', 'PROM1', 'ALDH1A1', 'ITGA6',

    # Immune checkpoints / tumor markers
    'PDCD1', 'CTLA4', 'HAVCR2', 'TIGIT', 'LAG3', 'CD274', 'PDCD1LG2',
    'HLA-A', 'HLA-B', 'HLA-C', 'HLA-DQA1', 'HLA-DPB1'
])))

hvg_genes = adata_normalized.var_names[adata_normalized.var['highly_variable']].tolist()
combined_genes = list(set(hvg_genes + nsclc_marker_genes))
genes_to_keep = [g for g in combined_genes if g in adata_normalized.var_names]
adata_normalized = adata_normalized[:, genes_to_keep]
adata_normalized

## Clustering

In [ ]:
sc.tl.pca(adata_normalized, svd_solver='arpack',n_comps=100)
# “elbow” — the point where additional PCs contribute little variance — and use that number in n_pcs.
sc.pl.pca_variance_ratio(adata_normalized, log=True, n_pcs=100)

In [ ]:
import scanpy as sc

# Parameters
NNei = 30
res = 0.01
nPCs = 30

# 1. Compute neighborhood graph
sc.pp.neighbors(adata_normalized, n_neighbors=NNei, n_pcs=nPCs)

print(f'Clustering with resolution = {res}')
# 2. Run Leiden clustering
leiden_key = f'nn_{NNei} leiden_{res}'
sc.tl.leiden(
    adata_normalized,
    resolution=res,
    key_added=leiden_key,
)
print(f'Clustering done, ploting...')

sc.pl.embedding(
    adata_normalized,
    basis="spatial",
    color=f'nn_{NNei} leiden_{res}',
    size=5,
    alpha=0.6,
    title=f"Spatial clusters (Leiden res={res})",
    show=False
)
plt.gca().invert_yaxis()
plt.show()



In [ ]:
print(f'Generating UMAP for resolution = {res}')

# 3. Compute UMAP embedding
sc.tl.umap(adata_normalized)

print(f'UMAP done, ploting...')

# 5. Plot UMAP colored by clustering result
sc.pl.umap(
    adata_normalized,
    color=leiden_key,
    title=f'nn_{NNei} Leiden {res}',
    size=10  # adjust based on dataset
)


## Annotation

In [ ]:
marker_dict = {
    'B cell': [
        'CD19', 'CD20', 'CD79A', 'CD79B','IGHM', 'MS4A1'
    ],
    'Basal cell': [
        'TP63', 'NGFR', 'MST1R'
    ],
    'CD4+ T cell': ['CD4'],
    'CD8+ T cell': ['CD8A'],
    'Cancer cell': [
        'EGFR', 'EPCAM', 'TTF-1', 'KRAS', 'TP53', 'NKX2-1',
        'TP63', 'CEACAM5', 'LOX'
    ],
    'Dendritic cell': [
        'CCL17', 'CD141', 'CD1A', 'CD1C', 'CD303', 'CD83'
    ],
    'Endothelial cell': [
        'CA4', 'CD31', 'CD34', 'CDH5', 'CLDN5', 'COL15A1', 'CPE',
        'DKK2', 'EDNRB', 'PECAM1', 'VWF', 'STEAP1', 'KDR', 'VEGFA'
    ],
    'Epithelial cell': [
        'AGER', 'CCDC153', 'CD326', 'CDH1', 'E-cadherin', 'EPCAM',
        'FOXJ1', 'KRT18', 'KRT19', 'KRT5', 'KRT7', 'KRT8', 'MUC5B',
        'SCGB3A2', 'SFTPB', 'SFTPC', 'TPPP3'
    ],
    'Fibroblast': ['COL1A1', 'COL1A2', 'PDGFRA', 'aSMA','FAP'],
    'Macrophage': [
        'APOE', 'CCL18', 'CCL2', 'CD11b', 'CD11c', 'CD14', 'CD163',
        'CD64', 'CD68', 'CD80', 'CD86', 'CSF1R', 'FN1', 'ITGAM',
        'MRC1', 'MS4A4A', 'SPP1', 'VCAN'
    ],
    'Mast cell': ['CPA3', 'KIT', 'TPSAB1', 'TPSB2'],
    'Monocyte': [
        'CD11b', 'CD14', 'CD163', 'CD68', 'FCN1', 'HLA-DR', 'IL1B', 'IL1R2'
    ],
    'Myeloid cell': ['CD68', 'LYZ'],
    'Natural killer cell': [
        'CD56', 'GNLY', 'KLRB1', 'KLRD1', 'KLRF1', 'NCAM1', 'NKG7'
    ],
    'Neutrophil': ['CD11b', 'CD15', 'CD16', 'CD66b', 'CSF3R', 'FCGR3B'],
    'Pericyte': ['PDGFRB'],
    'Stromal cell': ['ACTA2', 'PDGFRA'],
    'T cell': [
        'CD2', 'CD3', 'CD3D', 'CD3E', 'CD3G', 'CD4', 'CD8', 'CD8A',
        'CD8B', 'IL7R', 'LAG3', 'NKG7', 'TRAC'
    ],
    'Cancer stem cell': ['CD44', 'CD133', 'EpCAM', 'ALDH1A1']
}


In [ ]:
adata_normalized.obs.sample()

In [ ]:
import numpy as np
import scanpy as sc
import scipy.sparse

cluster_key = 'nn_30 leiden_0.01'

adata_combined = adata_normalized.copy()
# Step 0: Ensure sliceable matrix
if not isinstance(adata_combined.X, np.ndarray) and not scipy.sparse.isspmatrix_csr(adata_combined.X):
    adata_combined.X = adata_combined.X.tocsr()

# Step 1: Get all marker genes that exist in the data
all_marker_genes = sorted(set(g for genes in marker_dict.values() for g in genes))
present_markers = [g for g in all_marker_genes if g in adata_combined.var_names]

# Step 2: Compute variance across all cells
X = adata_combined[:, present_markers].X
if not isinstance(X, np.ndarray):
    X = X.toarray()  # convert sparse to dense if needed
gene_variance = X.var(axis=0)

# Step 3: Filter genes with variance above threshold
var_threshold = 0.001  # You can tune this
variable_genes = [g for g, v in zip(present_markers, gene_variance) if v > var_threshold]

# Step 4: Filter marker dict
filtered_marker_dict = {
    celltype: [g for g in genes if g in variable_genes]
    for celltype, genes in marker_dict.items()
}
filtered_marker_dict = {k: v for k, v in filtered_marker_dict.items() if len(v) > 0}

# Step 5: Plot
sc.pl.dotplot(
    adata_combined,
    var_names=filtered_marker_dict,
    groupby=cluster_key,
    standard_scale='var',  # Normalize each gene
    cmap='Reds',
    dendrogram=True,
    show=True
)


# DE clusters

In [ ]:
adata_normalized.obs.sample()

In [ ]:
import scanpy as sc

# Make sure your AnnData object has cluster labels
cluster_key = "nn_30 leiden_0.01"  # change to your actual key, e.g., "nn_30 leiden_0.5"
if cluster_key not in adata_normalized.obs:
    raise ValueError(f"{cluster_key} not found in adata.obs")

# Perform differential expression
sc.tl.rank_genes_groups(
    adata_normalized,
    groupby=cluster_key,
    method='t-test_overestim_var',  # or 'wilcoxon', 'logreg'
    use_raw=False,
    n_genes=100,
)

# Plot top DE genes
sc.pl.rank_genes_groups(adata_normalized, n_genes=20, sharey=False)
